<a href="https://colab.research.google.com/github/gretasam/gretasam/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instaliuojame reikalingas bibliotekas

In [1]:
!pip install openimages
!pip install torch torchvision

In [2]:
Class1 = 'Motorcycle'
Class2 = 'Balloon'
Class3 = 'Dog'

In [3]:
data_dir = 'data'

Siunčiamės nuotraukas

In [130]:
from openimages.download import download_dataset

data_dir = 'data'
classes = [Class1, Class2, Class3]
number_for_samples = 334

download_dataset(data_dir, classes, limit=number_for_samples)

100%|██████████| 334/334 [00:05<00:00, 57.04it/s]


{'motorcycle': {'images_dir': 'data/motorcycle/images'},
 'balloon': {'images_dir': 'data/balloon/images'},
 'dog': {'images_dir': 'data/dog/images'}}

Duomenų rinkinį paruošia klasifikacijai ir sukuria data loader, kuris ir yra naudojamas vertinimo metu

In [18]:
import torch
from torchvision import transforms, datasets

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root=data_dir, transform=preprocess)
#kiekvienai klasei sukuriamas skaitinis indeksas
class_to_idx = dataset.class_to_idx

#sukuriamas data loaderis, kuris automatizuotai tvarko duomenu paketus
#batch size reiskia, kiek vaizdu bus viename pakete
#workeriai reiskia, kad duomenu ikelimui naudojamas vienas procesas
batch_holder = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

Patikriname GPU ir sukuriame modelį, kurį naudosime nuotraukų vertinimui

In [19]:
from torchvision import models
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    raise SystemError("CUDA is not available. This script requires a GPU.")

#naudojami numatytieji svoriai, kurie buvo ismokti ant ImageNet duomenu rinkinio
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

#VGG16 yra apmokytas su 1000 ImageNet klasiu, todel paskutine sluoksnio dalis yra pakeiciamas
#i musu klasiu numeri
num_classes = 3
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, num_classes)

model.to(device)

model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Apskaičiuojame reikšmes
*  Tikslumas (Accuracy): Teisingai numatytų stebėjimų proporcija iš visų stebėjimų.
*  Atkūrimas (Recall): Tikrųjų teigiamų atvejų, kurie buvo teisingai identifikuoti, proporcija.
*   Precizija (Precision): Teigiamų identifikacijų, kurios iš tikrųjų buvo teisingos, proporcija.
*   F1 statistikos: Precizijos ir atpažinimo tikslumo svoris vidurkis.

In [20]:
import numpy as np

def calculate_metrics(ground_truth, predictions, threshold=0.5):
  predictions_thresholded = (predictions >= threshold).astype(np.float64)
  TP = np.sum((predictions_thresholded == 1) & (ground_truth == 1))
  TN = np.sum((predictions_thresholded == 0) & (ground_truth == 0))
  FP = np.sum((predictions_thresholded == 1) & (ground_truth == 0))
  FN = np.sum((predictions_thresholded == 0) & (ground_truth == 1))

  accuracy = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) > 0 else 0
  recall = TP / (TP + FN) if (TP + FN) > 0 else 0
  precision = TP / (TP + FP) if (TP + FP) > 0 else 0
  f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

  return {
    'accuracy': accuracy,
    'recall': recall,
    'precision': precision,
    'f1': f1
  }

Funkcija calculate_prob skirta apskaičiuoti tam tikros klasės tikimybes ir susieti jas su tikrąja šios klasės žyme
Išvedam sąrašą porų, kiekvienoje iš jų yra tikimybė ir dvejetainė tikroji vertė, nurodanti, ar klasės žymė atitinka laukiamą klasę.

In [21]:
def calculate_prob(arr2D, class_id, label_arr, class_label):
  result = []
  label_bool = 1.0
  for i in range(0, len(label_arr)):
    if class_label == label_arr[i]:
      label_bool = 1.0
    else:
      label_bool = 0.0
    result.append((arr2D[i][class_id], label_bool))
  return result

Funkcija extract_prob yra skirta ištraukti tikimybes iš duoto sąrašo.

In [22]:
def extract_prob(arr):
  result = []
  for prob, truth in arr:
      prob = prob.cpu().numpy() if prob.is_cuda else prob.numpy()
      result.append(prob)
  return result


extract_gt ištraukia tikrąsias vertes iš sąrašo

In [23]:
def extract_gt(arr):
  result = []
  for prob, truth in arr:
    result.append(truth)
  return result

Iškviečiam funkcijas ištraukti tikimybes bei tikrąsias vertes iš sąrašų
Kviečiame metrikų skaičiavimo funkciją

In [24]:
def extractAndCalculate(gt_prob_list, treshold):
  gt_list = extract_gt(gt_prob_list)
  pred_list = extract_prob(gt_prob_list)
  return calculate_metrics(np.array(gt_list), np.array(pred_list), treshold)


Funkcija randa geriausią threshold kiekvienai klasei

In [25]:
import numpy as np
from sklearn.metrics import f1_score

def find_best_threshold_for_class(probabilities, ground_truths):
    thresholds = np.arange(0.0, 1, 0.05)
    best_threshold = 0.5
    best_f1 = 0.0

    for threshold in thresholds:
        predictions = (probabilities >= threshold).astype(int)
        f1 = f1_score(ground_truths, predictions)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1

In [26]:
def apply_threshold(probabilities, threshold):
    return (probabilities >= threshold).astype(int)

In [27]:
class1_id = class_to_idx[Class1.lower()]
class2_id = class_to_idx[Class2.lower()]
class3_id = class_to_idx[Class3.lower()]

class1_probabilities = []
class2_probabilities = []
class3_probabilities = []

class1_ground_truths = []
class2_ground_truths = []
class3_ground_truths = []

Kiekvienas rinkinys (batch) siunčiamas į modelį, kuris kiekvienai nuotraukai atiduoda 1000 ilgio sąrašą: [tikimybė_kad_nuotraukoje_klasė1, ... ,  tikimybė_kad_nuotraukoje_klasė1000]

Skaičiuojami tikslumas (accuracy), atkūrimas (recall), precizija (precision) ir F1 statistikos kiekvienai klasei atskirai

In [28]:
import torch

for images, labels in batch_holder:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    probabilities = torch.sigmoid(outputs).detach()

    probabilities_np = probabilities.cpu().numpy()
    labels_np = labels.cpu().numpy()

    class1_probabilities.extend(probabilities_np[:, class1_id])
    class2_probabilities.extend(probabilities_np[:, class2_id])
    class3_probabilities.extend(probabilities_np[:, class3_id])

      # Corrected line for VGG16
    num_classes = model.classifier[-1].out_features
    #num_classes = model.fc.out_features
    one_hot_labels = np.eye(num_classes)[labels_np]

    class1_ground_truths.extend(one_hot_labels[:, class1_id])
    class2_ground_truths.extend(one_hot_labels[:, class2_id])
    class3_ground_truths.extend(one_hot_labels[:, class3_id])

class1_probabilities = np.array(class1_probabilities)
class2_probabilities = np.array(class2_probabilities)
class3_probabilities = np.array(class3_probabilities)

class1_ground_truths = np.array(class1_ground_truths)
class2_ground_truths = np.array(class2_ground_truths)
class3_ground_truths = np.array(class3_ground_truths)


#print(len(class1_probabilities), len(class1_ground_truths))
#print(len(class2_probabilities), len(class2_ground_truths))
#print(len(class3_probabilities), len(class3_ground_truths))



best_threshold_class1, best_f1_class1 = find_best_threshold_for_class(class1_probabilities, class1_ground_truths)
best_threshold_class2, best_f1_class2 = find_best_threshold_for_class(class2_probabilities, class2_ground_truths)
best_threshold_class3, best_f1_class3 = find_best_threshold_for_class(class3_probabilities, class3_ground_truths)

print(f"Best threshold for {Class1}: {best_threshold_class1} with F1: {best_f1_class1}")
print(f"Best threshold for {Class2}: {best_threshold_class2} with F1: {best_f1_class2}")
print(f"Best threshold for {Class3}: {best_threshold_class3} with F1: {best_f1_class3}")

#threshold keitimas kiekvienai klasei
#best_threshold_class1 = 0.8
#best_threshold_class2 = 0.8
#best_threshold_class3 = 0.8

class1_metrics = calculate_metrics(class1_ground_truths, apply_threshold(class1_probabilities, best_threshold_class1), best_threshold_class1)
class2_metrics = calculate_metrics(class2_ground_truths, apply_threshold(class2_probabilities, best_threshold_class2), best_threshold_class2)
class3_metrics = calculate_metrics(class3_ground_truths, apply_threshold(class3_probabilities, best_threshold_class3), best_threshold_class3)

print(f"{Class1} statistics with best threshold: {class1_metrics}")
print(f"{Class2} statistics with best threshold: {class2_metrics}")
print(f"{Class3} statistics with best threshold: {class3_metrics}")

Best threshold for Motorcycle: 0.2 with F1: 0.500374531835206
Best threshold for Balloon: 0.5 with F1: 0.5209205020920503
Best threshold for Dog: 0.0 with F1: 0.5
Motorcycle statistics with best threshold: {'accuracy': 0.3343313373253493, 'recall': 1.0, 'precision': 0.3336663336663337, 'f1': 0.500374531835206}
Balloon statistics with best threshold: {'accuracy': 0.5429141716566867, 'recall': 0.7455089820359282, 'precision': 0.4003215434083601, 'f1': 0.5209205020920503}
Dog statistics with best threshold: {'accuracy': 0.3333333333333333, 'recall': 1.0, 'precision': 0.3333333333333333, 'f1': 0.5}
